In [1]:
from sentence_transformers import SentenceTransformer, util
import joblib
import numpy as np
import pandas as pd


In [2]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
model = joblib.load('questions-categorizer-v2-KNeighborsClassifier.model.joblib')

C:\Python311\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [47]:
# Initialize fixed size dataframe to setup cache (LFU)
num_rows = 4  # Size of the cache
columns = ['Question', 'Response', 'Access Count']
# Create a list of dictionaries with default values
data = [{'Question': '', 'Response': '', 'Access Count': 0} for _ in range(num_rows)]
# Create a DataFrame from the list of dictionaries
cache_for_VM = pd.DataFrame(data, columns=columns)

print(cache_for_VM)


  Question Response  Access Count
0                               0
1                               0
2                               0
3                               0


In [92]:
# New values for the row
new_values = {'Question': 'How does TLB caching improve virtual memory performance?', 'Response': 'Resp 23', 'Access Count': 1}

# Change the values using .loc[]
cache_for_VM.loc[1] = new_values

print(cache_for_VM.iloc[1][1])

Resp 23


In [75]:
# encoded cache files
embeddings_cache_for_VM = embedder.encode(cache_for_VM['Question'])
print(len(embeddings_cache_for_VM[1]))

384


In [103]:
def give_the_response(new_request, category):
  encoded_new_request = embedder.encode(new_request)

  response = cache_handler(new_request, encoded_new_request, category)
  return response


In [104]:
def cache_handler(request, encoded_request, category):
  if category == "vm":
    cos_sim = util.cos_sim(embeddings_cache_for_VM, encoded_request)
    if (max(cos_sim) > 0.75):
      print("* * * * * * * From Cache * * * * * * * ")
      print(cos_sim.argmax().item())
      return cache_for_VM.iloc[cos_sim.argmax().item()][1]
    else:
      return call_API(request)
  # add other categories


In [77]:
def call_API(request):
  print("* * * * * * * Calling API * * * * * * * ")
  mock_resp = "API response for -> " + request
  return mock_resp


In [101]:
test_sentence = "can you please tell me how can we improve performances of cache using TLB?"

In [105]:
category = "vm"
response_for_test_sentence = give_the_response(test_sentence, category)
print(response_for_test_sentence)

* * * * * * * From Cache * * * * * * * 
1
Resp 23
